# Train model (Random Forest) and parameters tuning

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline


from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate

In [2]:
#read data from dataset (after feature extraction )
# train model: aftcat, CV(Pipeline(imbalanced, MinMaxScaler, selector, model))
data_aftcat = pd.read_csv(r"./2-1 5388 data_aftcat.csv")
data_aftcat_y = pd.read_csv(r"./2-1 5388 data_aftcat label.csv")

In [122]:
data_aftcat.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,7496,43,1,1,0,6,0,0,0.000000,0.000000,...,0,40,0.0,0.0,0,0,0.0,0.0,0,0
1,46564,61,1,1,6,6,6,6,6.000000,0.000000,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
2,80,69,2,0,0,0,0,0,0.000000,0.000000,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
3,80,98403702,7,5,377,11595,377,0,53.857143,142.492606,...,1,32,986.0,0.0,986,986,98400000.0,0.0,98400000,98400000
4,53,30843,1,1,54,70,54,54,54.000000,0.000000,...,0,40,0.0,0.0,0,0,0.0,0.0,0,0


In [3]:
# ---------------- transform all feature with MinMaxScaler()---------------- 
mm = MinMaxScaler()
data_aftcat_aftmm = pd.DataFrame(mm.fit_transform(data_aftcat))
data_aftcat_aftmm.columns = mm.get_feature_names_out()

In [4]:
data_aftcat_aftmm.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.114506,3.666672e-07,0.000000,0.000004,0.000000,1.042243e-08,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.666667,0.00000,0.0,0.00000,0.00000,0.00,0.0,0.00,0.00
1,0.711292,5.166674e-07,0.000000,0.000004,0.000005,1.042243e-08,0.000516,0.003026,0.002068,0.000000,...,0.000000,0.333333,0.00000,0.0,0.00000,0.00000,0.00,0.0,0.00,0.00
2,0.001222,5.833341e-07,0.000005,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.333333,0.00000,0.0,0.00000,0.00000,0.00,0.0,0.00,0.00
3,0.001222,8.200320e-01,0.000031,0.000019,0.000324,2.014134e-05,0.032411,0.000000,0.018560,0.024587,...,0.000005,0.533333,0.00001,0.0,0.00001,0.00001,0.82,0.0,0.82,0.82
4,0.000810,2.570337e-04,0.000000,0.000004,0.000046,1.215950e-07,0.004642,0.027231,0.018609,0.000000,...,0.000000,0.666667,0.00000,0.0,0.00000,0.00000,0.00,0.0,0.00,0.00


In [5]:
# -------------------------functions--------------
# function score_mean_std: calculate the average and std of scores
def score_mean_std(score_sf):
    index_a1 = []
    index_a2 = []

    for i in range(score_sf.shape[1]):
        index_a1.append(np.average(score_sf[score_sf.columns[i]]))
        index_a2.append(np.std(score_sf[score_sf.columns[i]]))

    index_a1 = pd.DataFrame(index_a1, index = score_sf.columns)
    index_a2 = pd.DataFrame(index_a2, index = score_sf.columns)
    #print(index_a1.T)
    #print(index_a2.T)
    re = pd.concat([index_a1.T,index_a2.T],axis=0)
    re.index = ["avg","std"]
    #print(re)
    return re

In [6]:
def train_model(sm, se, model, X, y):
    #print(model, "\n")
    #pipe_steps = [('smo',sm),('mm',m),('selector',se),('model',model)]
    pipe_steps = [('smo',sm),('selector',se),('model',model)]
    id_pipeline = Pipeline(steps=pipe_steps)

    # evaluate the pipeline using the crossvalidation technique defined in cv

    # ------------------- score ------------------------------------
    scoring = {'f1', 'precision', 'accuracy',
            'recall', 'roc_auc'}

    score_sf = cross_validate(id_pipeline, X, y.values.ravel(),cv=10,scoring=scoring, n_jobs=-1)
    score_sf_re = score_mean_std(pd.DataFrame(score_sf))

    return score_sf,score_sf_re

## train model with default parameters

In [7]:
smo = SMOTE()
selector = VarianceThreshold(np.median(data_aftcat_aftmm.var().values))
#model_rf = RandomForestClassifier(random_state = 90, min_samples_split=2,n_estimators=61,max_depth=24, max_features=27,min_samples_leaf=1, n_jobs=-1)
#model_grb = GradientBoostingClassifier(max_features=3,learning_rate=0.1,n_estimators=130,min_samples_split=100,min_samples_leaf=7,max_depth=15,random_state = 10)
model_rf = RandomForestClassifier(random_state = 90, n_jobs=-1)

In [8]:
# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(smo, selector, model_rf, data_aftcat_aftmm, data_aftcat_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(n_jobs=-1, random_state=90)
score_sf_re_rf: 
      fit_time  score_time  test_roc_auc  test_accuracy   test_f1  \
avg  16.80007    0.169732      0.999592       0.997899  0.988424   
std   3.64961    0.012313      0.000800       0.000953  0.005250   

     test_precision  test_recall  
avg        0.986287     0.990637  
std        0.007091     0.008315  


## Round #1 n_estimators

In [9]:
print(model_rf)
scorel = []
for i in range(1,200):
    model_rf = RandomForestClassifier(n_estimators=i,n_jobs=-1,random_state = 90)
    score_sf_rf,score_sf_re_rf = train_model(smo, selector, model_rf, data_aftcat_aftmm, data_aftcat_y)
    #print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
    scorel.append(np.array(score_sf_rf['test_f1']).mean())
    #print(np.array(score_sf_rf['test_f1']).mean())
print(max(scorel), scorel.index(max(scorel))+1)

RandomForestClassifier(n_jobs=-1, random_state=90)
0.9904151126906287 115


## Round #2 max_depth

In [10]:
scorel = []
for i in range(1,40):
    #print(i)
    model_rf = RandomForestClassifier(max_depth=i, n_estimators=115,n_jobs=-1,random_state = 90)
    #print(model_rf)
    score_sf_rf,score_sf_re_rf = train_model(smo, selector, model_rf, data_aftcat_aftmm, data_aftcat_y)
    #print(score_sf_rf)
    scorel.append(np.array(score_sf_rf['test_f1']).mean())
    #print("max scorel:",max(scorel))
    #print(scorel.index(max(scorel))+1)
print(max(scorel), scorel.index(max(scorel))+1)    

0.9899795639098166 23


## Round #3 max_features

In [11]:
scorel = []
for i in range(1,30):
    model_rf = RandomForestClassifier(max_features=i, max_depth=23, n_estimators=115,n_jobs=-1,random_state = 90)
    score_sf_rf,score_sf_re_rf = train_model(smo, selector, model_rf, data_aftcat_aftmm, data_aftcat_y)
    scorel.append(np.array(score_sf_rf['test_f1']).mean())
print(max(scorel), scorel.index(max(scorel))+1)   

0.9924246185965855 17


## Round #4 min_samples_leaf

In [12]:
scorel = []
for i in range(1,30):
    model_rf = RandomForestClassifier(min_samples_leaf=i,max_features=17, max_depth=23, n_estimators=115,n_jobs=-1,random_state = 90)
    score_sf_rf,score_sf_re_rf = train_model(smo, selector, model_rf, data_aftcat_aftmm, data_aftcat_y)
    scorel.append(np.array(score_sf_rf['test_f1']).mean())
print(max(scorel), scorel.index(max(scorel))+1)   

0.9908665440963077 1


## Round #5 min_samples_split

In [13]:
scorel = []
for i in range(2,15):
    #print(i)
    model_rf = RandomForestClassifier(min_samples_split=i, min_samples_leaf=1, max_features=17, max_depth=23, n_estimators=115,n_jobs=-1,random_state = 90)
    #print(model_rf)
    score_sf_rf,score_sf_re_rf = train_model(smo, selector, model_rf, data_aftcat_aftmm, data_aftcat_y)
    #print(score_sf_rf)
    scorel.append(np.array(score_sf_rf['test_f1']).mean())
print(max(scorel), scorel.index(max(scorel))+1)  

0.9917594301265188 1


## Round #6 criterion

In [14]:
model_rf = RandomForestClassifier(criterion='gini',min_samples_split=2, min_samples_leaf=1, max_features=17, max_depth=23, n_estimators=115,n_jobs=-1,random_state = 90)
score_sf_rf,score_sf_re_rf = train_model(smo, selector, model_rf, data_aftcat_aftmm, data_aftcat_y)
print(score_sf_rf,"\n")
print("score_sf_re_rf: \n",score_sf_re_rf)

{'fit_time': array([40.82344675, 41.11998391, 40.48574305, 40.27172995, 34.71252728,
       35.94078994, 35.71828103, 35.87124681, 20.31750798, 19.28667903]), 'score_time': array([0.14280391, 0.17595887, 0.13775373, 0.13632512, 0.13632083,
       0.13877106, 0.1453948 , 0.14659023, 0.17603683, 0.24943089]), 'test_roc_auc': array([0.99748543, 0.99992563, 0.9999405 , 0.99992464, 0.99978383,
       0.99980655, 0.99995933, 0.99999603, 0.99989484, 0.99993849]), 'test_accuracy': array([0.9959596 , 0.99919192, 0.9979798 , 0.99878788, 0.9979798 ,
       0.99878739, 0.99878739, 0.99878739, 0.99878739, 0.99919159]), 'test_f1': array([0.97777778, 0.99555556, 0.98886414, 0.99331849, 0.98886414,
       0.99334812, 0.99328859, 0.99334812, 0.99328859, 0.99553571]), 'test_precision': array([0.97777778, 0.99115044, 0.98666667, 0.99111111, 0.98666667,
       0.98678414, 0.9955157 , 0.98678414, 0.9955157 , 0.99553571]), 'test_recall': array([0.97777778, 1.        , 0.99107143, 0.99553571, 0.99107143,
   

In [15]:
model_rf = RandomForestClassifier(criterion='entropy',min_samples_split=2, min_samples_leaf=1, max_features=17, max_depth=23, n_estimators=115,n_jobs=-1,random_state = 90)
score_sf_rf,score_sf_re_rf = train_model(smo, selector, model_rf, data_aftcat_aftmm, data_aftcat_y)
print(score_sf_rf,"\n")
print("score_sf_re_rf: \n",score_sf_re_rf)

{'fit_time': array([31.04611421, 31.48563576, 31.26496005, 31.7279799 , 32.8314271 ,
       33.61550593, 33.41234779, 33.54063797, 15.93730116, 15.39619493]), 'score_time': array([0.12671995, 0.13765597, 0.14214182, 0.17104006, 0.13407993,
       0.17059016, 0.16328406, 0.19950986, 0.12885118, 0.09481096]), 'test_roc_auc': array([0.99751704, 0.99992761, 0.99994546, 0.99992067, 0.99982944,
       0.9996994 , 0.99996528, 0.99999603, 0.99986409, 0.99991071]), 'test_accuracy': array([0.99676768, 0.99919192, 0.99838384, 0.99919192, 0.99838384,
       0.99797898, 0.99919159, 0.99878739, 0.99838319, 0.99878739]), 'test_f1': array([0.98214286, 0.99555556, 0.99111111, 0.99555556, 0.99107143,
       0.98891353, 0.99553571, 0.99331849, 0.99107143, 0.99331849]), 'test_precision': array([0.98654709, 0.99115044, 0.98672566, 0.99115044, 0.99107143,
       0.98237885, 0.99553571, 0.99111111, 0.99107143, 0.99111111]), 'test_recall': array([0.97777778, 1.        , 0.99553571, 1.        , 0.99107143,
   

# model_rf = RandomForestClassifier(criterion='entropy',min_samples_split=2, min_samples_leaf=1, max_features=17, max_depth=23, n_estimators=115,n_jobs=-1,random_state = 90)